In [1]:
import crocoddyl
import pinocchio
from IPython.display import HTML
import mim_solvers
import numpy as np

In [2]:
from matplotlib import animation
from matplotlib import pyplot as plt


def animatePointMass(xs, sleep=50, show=False):
    print("processing the animation ... ")
    mass_size = 1.0
    fig = plt.figure()
    ax = plt.axes(xlim=(-2, 12), ylim=(-5, 5))
    patch = plt.Circle((0, 0), radius=0.2, fc="b")
    obstacle = plt.Circle((5, 0), radius=2, fc="k")
    time_text = ax.text(0.02, 0.95, "", transform=ax.transAxes)

    def init():
        ax.add_patch(patch)
        ax.add_patch(obstacle)
        ax.set_aspect('equal', adjustable='box')
        time_text.set_text("")
        return patch, time_text

    def animate(i):
        x_pm = xs[i][0]
        y_pm = xs[i][1]
        patch.set_center((x_pm, y_pm))
        time = i * sleep / 1000.0
        time_text.set_text(f"time = {time:.1f} sec")
        return patch, time_text

    anim = animation.FuncAnimation(
        fig, animate, init_func=init, frames=len(xs), interval=sleep, blit=True
    )
    print("... processing done")
    if show:
        plt.show()
    return anim

In [3]:
class DifferentialActionModelPointMass(crocoddyl.DifferentialActionModelAbstract):
    def __init__(self):
        crocoddyl.DifferentialActionModelAbstract.__init__(
            self, crocoddyl.StateVector(4), 2, 3
        )  # nu = 2 {Fx, Fy}; nr = 3 {XReg, UReg, Obs}
        
        self.unone = np.zeros(self.nu)
        self.m = 1.0
        self.costWeights = np.zeros(self.nr)

    def calc(self, data, x, u=None):
        if u is None:
            u = self.unone
        # Getting the state and control variables
        X, Y, Xdot, Ydot = x[0], x[1], x[2], x[3]
        fx = u[0]
        fy = u[1]

        # Shortname for system parameters
        m = self.m

        # Defining the equation of motions
        Xddot = fx / m
        Yddot = fy / m
        data.xout = np.matrix([Xddot, Yddot]).T

        # Cost parameters (Translation + Obstacle)
        target = np.array([10, 0, 0, 0])
        obs_center = np.array([5, 0])
        obs_rad = 2.0
        obs_act = 1.0

        trans_r = np.linalg.norm(target - x)
        obs_d = np.sqrt(np.sum((x[:2] - obs_center)*(x[:2] - obs_center))) - obs_rad
        if obs_d > obs_act:
            obs_r = 0
        else:
            obs_r = np.linalg.norm(obs_d - obs_act)
        u_r = np.linalg.norm(u)
        # Computing the cost residual and value
        data.r = np.matrix(self.costWeights * np.array([trans_r, u_r, obs_r])).T
        # data.r = np.matrix(self.costWeights * np.array([trans_r])).T
        data.cost = 0.5 * sum(np.asarray(data.r) ** 2)

    # def calcDiff(self, data, x, u=None):
    #     if u is None:
    #         u = self.unone
    #     data.Fx[:,:] = np.array([0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0])
    #     data.Fu[:,:] = np.array([self.m, 0.0], [0.0, self.m])
    #     data.Lx[:] = 
    #     data.Lu[:] = 
    #     data.Lxx[:,:] =
    #     data.Luu[:,:] = 
    #     data.Lux[:,:] = 


In [4]:
PM_DAM_running = DifferentialActionModelPointMass()
PM_DAM_terminal = DifferentialActionModelPointMass()
pmDataRunning = PM_DAM_running.createData()
pmDataTerminal = PM_DAM_terminal.createData()
x = np.zeros(4)
u = np.zeros(2)
PM_DAM_running.calc(pmDataRunning, x, u)

In [5]:
PM_DAM_running.costWeights = np.array([10, 10, 0])
PM_DAM_terminal.costWeights = np.array([100, 10, 0])

In [6]:
PM_ND_R = crocoddyl.DifferentialActionModelNumDiff(PM_DAM_running, False)
PM_ND_T = crocoddyl.DifferentialActionModelNumDiff(PM_DAM_terminal, False)

In [7]:
timeStep = 5e-2
PM_IAM = crocoddyl.IntegratedActionModelEuler(PM_ND_R, timeStep)
PM_IAM_T = crocoddyl.IntegratedActionModelEuler(PM_ND_T, 0.0)

In [8]:
x0 = np.array([0.0, 0.0, 0.0, 0.0])
u0 = np.array([0.0, 0.0])
T = 100
problem = crocoddyl.ShootingProblem(x0, [PM_IAM] * T, PM_IAM_T)

In [ ]:
# Creating the SQP solver
sqp = mim_solvers.SolverSQP(problem)
sqp.setCallbacks([crocoddyl.CallbackVerbose()])
sqp.with_callbacks=True
sqp.termination_tolerance = 5e-4
xs_init = [x0 for i in range(T+1)]
us_init = [u0 for i in range(T)]

# Solving this problem
done = sqp.solve(xs_init, us_init, 100)
print(done)
print(sqp.us)

In [ ]:
# HTML(animatePointMass(sqp.xs).to_jshtml())

In [16]:
def traj_cost(xs, us, running_w, terminal_w, dt):
    cost = 0
    target = np.array([10, 0, 0, 0])
    obs_center = np.array([5, 0])
    obs_rad = 2.0
    obs_act = 1.0
    for X, U in zip(xs[:-1], us):
        trans_r = np.linalg.norm(target - X)
        obs_d = np.sqrt(np.sum((X[:2] - obs_center)*(X[:2] - obs_center))) - obs_rad
        if obs_d > obs_act:
            obs_r = 0
        else:
            obs_r = np.linalg.norm(obs_d - obs_act)
        u_r = np.linalg.norm(U)
        r = np.matrix(running_w * np.array([trans_r, u_r, obs_r])).T
        cost += 0.5 * sum(np.asarray(r) ** 2) * dt
    X = xs[-1]
    trans_r = np.linalg.norm(target - X)
    obs_d = np.sqrt(np.sum((X[:2] - obs_center)*(X[:2] - obs_center))) - obs_rad
    if obs_d > obs_act:
        obs_r = 0
    else:
        obs_r = np.linalg.norm(obs_d - obs_act)
    r = np.matrix(terminal_w * np.array([trans_r, obs_r])).T
    cost+=0.5 * sum(np.asarray(r) ** 2)
    return cost

def traj_cost_derivative(xs, us, w):
    
    m = 1
    Fx = np.array(
        [0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0]
    )
    Fu = np.array(
        [m, 0.0],
        [0.0, m]
    )
    Lx = np.array(
        [(xs[0]-target[0])*w[0]**2 + (xs[0]-obs_center[0])*w[2]**2],
        [(xs[1]-target[1])*w[0]**2 + (xs[1]-obs_center[1])*w[2]**2],
        [xs[2]*w[0]**2],
        [xs[3]*w[0]**2]
    )
    Lu = np.array(
        [u[0]*w[1]**2],
        [u[1]*w[1]**2]
    )
    Lxx = np.array(
        [w[0]**2+w[2]**2, 0.0, 0.0, 0.0],
        [0.0, w[0]**2+w[2]**2, 0.0, 0.0],
        [0.0, 0.0, w[0]**2, 0.0],
        [0.0, 0.0, 0.0, w[0]**2]
    )
    Luu = np.array(
        [w[1]**2, 0.0],
        [0.0, w[1]**2]
    )
    

    